In [ ]:
!pip install pytorch-lightning

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.0/823.0 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 96.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 57.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision import transforms
import wandb
import pytorch_lightning as pl
from torchvision import transforms
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from pytorch_lightning.loggers import WandbLogger

In [ ]:
# Define data transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Define the paths to the training and validation directories
train_dir = '/content/drive/MyDrive/DA6401_Assignment-2/inaturalist_12K/train'         # Add appropiate path
val_dir = '/content/drive/MyDrive/DA6401_Assignment-2/inaturalist_12K/val'

In [ ]:
# Create the training and validation datasets
train_dataset = ImageFolder(train_dir, transform=data_transforms)
val_dataset = ImageFolder(val_dir, transform=data_transforms)

# Create the data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4)

wandb.login()

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mdkarimullahaque (mdkarimullahaque-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
class Mish(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return x * torch.tanh(F.softplus(x))


class CNN(pl.LightningModule):
    def __init__(self, input_shape, num_classes, num_filters=32, filter_size=(3, 3), activation='relu',
                 dense_neurons=128, dropout_rate=0.2, data_augmentation=False, batch_norm=True,
                 filter_organization='same'):
        super().__init__()
        self.input_shape = input_shape
        self.num_classes = num_classes
        self.num_filters = num_filters
        self.filter_size = filter_size
        self.activation = activation
        self.dense_neurons = dense_neurons
        self.dropout_rate = dropout_rate
        self.data_augmentation = data_augmentation
        self.batch_norm = batch_norm
        self.filter_organization = filter_organization

        # Define data augmentation transformations
        self.data_augmentation_transforms = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(degrees=15),
            # Add other desired transformations here
        ])

        # Define the layers
        self.conv_layers = self._create_conv_layers()
        self.dense_layers = self._create_dense_layers()

    def _create_conv_layers(self):
        layers = []
        in_channels = self.input_shape[0]
        out_channels = self.num_filters

        if self.filter_organization == 'double':
            out_channels_list = [out_channels] + [out_channels * 2**i for i in range(1, 5)]
        elif self.filter_organization == 'halve':
            out_channels_list = [out_channels * 2**i for i in range(4, -1, -1)]
        else:
            out_channels_list = [out_channels] * 5

        for out_channels in out_channels_list:
            layers.append(nn.Conv2d(in_channels, out_channels, self.filter_size, padding='same'))
            if self.batch_norm:
                layers.append(nn.BatchNorm2d(out_channels))
            layers.append(self.get_activation())
            layers.append(nn.MaxPool2d(2))
            in_channels = out_channels

        return nn.Sequential(*layers)

    def _create_dense_layers(self):
        layers = [
            nn.Flatten(),
            nn.Linear(self.conv_output_size(), self.dense_neurons),
            self.get_activation(),
            nn.Dropout(self.dropout_rate),
            nn.Linear(self.dense_neurons, self.num_classes)
        ]
        return nn.Sequential(*layers)

    def forward(self, x):
        if self.data_augmentation:
            # Apply data augmentation to the input batch
            augmented_batch = []
            for image in x:
                augmented_image = self.data_augmentation_transforms(image)
                augmented_batch.append(augmented_image)
            x = torch.stack(augmented_batch)

        x = self.conv_layers(x)
        x = self.dense_layers(x)
        return x

    def get_activation(self):
        if self.activation == 'relu':
            return nn.ReLU()
        elif self.activation == 'gelu':
            return nn.GELU()
        elif self.activation == 'silu':
            return nn.SiLU()
        elif self.activation == 'mish':
            return Mish()
        else:
            raise ValueError("Invalid activation function specified.")

    def conv_output_size(self):
        dummy_input = torch.zeros(1, *self.input_shape)
        dummy_output = self.conv_layers(dummy_input)
        return dummy_output.flatten(1).size(-1)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        accuracy = (outputs.argmax(dim=1) == labels).float().mean()
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        self.log('train_accuracy', accuracy, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = F.cross_entropy(outputs, labels)
        accuracy = (outputs.argmax(dim=1) == labels).float().mean()
        self.log('val_loss', loss, on_epoch=True, prog_bar=True, logger=True)
        self.log('val_accuracy', accuracy, on_epoch=True, prog_bar=True, logger=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer


sweep_config = {
    'method': 'random',  # Specify the search method
    'parameters': {
        'num_filters': {'values': [32, 64, 128]},
        'activation': {'values': ['relu', 'gelu', 'silu', 'mish']},
        'data_augmentation': {'values': [True, False]},
        'batch_norm': {'values': [True, False]},
        'dropout': {'values': [0.2, 0.3]},
        'filter_organization': {'values': ['same', 'double', 'halve']}
    }
}


sweep_id = wandb.sweep(sweep=sweep_config, project="DL_Assignment_2_try_1")

def train_model():
    with wandb.init() as run:

        run_name = f"num_filters_{wandb.config.num_filters}_activation_{wandb.config.activation}_data_augmentation_{wandb.config.data_augmentation}_batch_norm_{wandb.config.batch_norm}_dropout_{wandb.config.dropout}_filter_organization_{wandb.config.filter_organization}"
        wandb.run.name = run_name
        config = wandb.config

        # Create the training and validation datasets
        train_dataset = ImageFolder(train_dir, transform=data_transforms)
        val_dataset = ImageFolder(val_dir, transform=data_transforms)

        # Create the data loaders
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
        val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)
        input_shape = (3, 224, 224)
        num_classes = len(train_dataset.classes)
        model = CNN(input_shape, num_classes, num_filters=config.num_filters, activation=config.activation,
                    data_augmentation=config.data_augmentation, batch_norm=config.batch_norm,
                    dropout_rate=config.dropout, filter_organization=config.filter_organization)
        model = model.to('cuda')

        # Initialize wandb logger
        wandb_logger = WandbLogger()
        # Train the model
        trainer = pl.Trainer(max_epochs=10,accelerator='gpu', devices=1, logger=wandb_logger)
        trainer.fit(model, train_loader, val_loader)

wandb.agent(sweep_id, function=train_model, count = 50)

Create sweep with ID: vu5bt38j
Sweep URL: https://wandb.ai/mdkarimullahaque-iit-madras/DL_Assignment_2_try_1/sweeps/vu5bt38j


wandb: Agent Starting Run: jzui02rh with config:
wandb: 	activation: relu
wandb: 	batch_norm: True
wandb: 	data_augmentation: True
wandb: 	dropout: 0.2
wandb: 	filter_organization: same
wandb: 	num_filters: 128


INFO:pytorch_lightning.utilities.rank_zero:You are using the plain ModelCheckpoint callback. Consider using LitModelCheckpoint which with seamless uploading to Model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type       | Params | Mode 
----------------------------------------------------
0 | conv_layers  | Sequential | 595 K  | train
1 | dense_layers | Sequential | 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

wandb: ERROR Run musibbsu errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-6-854ab2ce261e>", line 154, in train_model
wandb: ERROR     model = model.to('cuda')
wandb: ERROR             ^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/lightning_fabric/utilities/device_dtype_mixin.py", line 55, in to
wandb: ERROR     return super().to(*args, **kwargs)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1343, in to
wandb: ERROR     return self._apply(convert)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 903, in _apply
wandb: ERROR     module._apply(fn)
wandb: ERROR   File "/usr/local/lib/py

Traceback (most recent call last):
  File "<ipython-input-6-854ab2ce261e>", line 154, in train_model
    model = model.to('cuda')
            ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/lightning_fabric/utilities/device_dtype_mixin.py", line 55, in to
    return super().to(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1343, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 903, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 903, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 930, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1329, in conve

wandb: ERROR Run zhattxpr errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-6-854ab2ce261e>", line 154, in train_model
wandb: ERROR     model = model.to('cuda')
wandb: ERROR             ^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/lightning_fabric/utilities/device_dtype_mixin.py", line 55, in to
wandb: ERROR     return super().to(*args, **kwargs)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1343, in to
wandb: ERROR     return self._apply(convert)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 903, in _apply
wandb: ERROR     module._apply(fn)
wandb: ERROR   File "/usr/local/lib/py

Traceback (most recent call last):
  File "<ipython-input-6-854ab2ce261e>", line 154, in train_model
    model = model.to('cuda')
            ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/lightning_fabric/utilities/device_dtype_mixin.py", line 55, in to
    return super().to(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1343, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 903, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 903, in _apply
    module._apply(fn)
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 930, in _apply
    param_applied = fn(param)
                    ^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1329, in conve

wandb: ERROR Run wenku0gt errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR   File "<ipython-input-6-854ab2ce261e>", line 154, in train_model
wandb: ERROR     model = model.to('cuda')
wandb: ERROR             ^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/lightning_fabric/utilities/device_dtype_mixin.py", line 55, in to
wandb: ERROR     return super().to(*args, **kwargs)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 1343, in to
wandb: ERROR     return self._apply(convert)
wandb: ERROR            ^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py", line 903, in _apply
wandb: ERROR     module._apply(fn)
wandb: ERROR   File "/usr/local/lib/py